<a href="https://colab.research.google.com/github/JoaoAssalim/Kaggle-Projects/blob/main/sentiment_analysis_tensorflow_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaggle Project: ❤️ vs 😡: Sentiment Analysis 📝

Link to page: https://www.kaggle.com/datasets/mohidabdulrehman/vs-sentiment-analysis/data

In [ ]:
!python -m spacy download en_core_web_sm

In [124]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding, LSTM, Dense, Input, Dropout, Bidirectional
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import nltk
from nltk.corpus import stopwords

import pandas as pd
import spacy

In [16]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
df = pd.read_csv("/content/Beginner_Reviews_dataset.csv")
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,sentence,label
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [18]:
### In this step, I made an function that tokenize all the data and
### lemmatize to get the radical from all the words and remove stopwords
### to make the sentences more clean

nlp = spacy.load("en_core_web_sm")

def custom_lemmatizer(text):
    doc = nlp(text)
    lemmatized_words = [token.lemma_.lower() for token in doc if token.lemma_ not in stop_words and len(token.lemma_) > 1]
    return " ".join(lemmatized_words)

In [19]:
df["sentence"] = df["sentence"].apply(custom_lemmatizer)

In [53]:
df = df.sample(frac=1)

In [54]:
df.head()

,sentence,label
108,go back next trip,1
968,get much well service pizza place next door se...,0
140,,0
870,start review two star edit give one,0
252,greek dressing creamy flavorful,1


In [97]:
X = df['sentence']
y = df['label']

In [116]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

> Output Modes:

* output_mode='int':
    ```
    Mapeia cada palavra para um índice inteiro.
    O texto "Eu gosto de programar em Python" pode
    ser transformado em algo como [2, 3, 4, 5, 6].
    ```

* output_mode='binary':
    ```
    Produz um vetor binário indicando a presença
    ou ausência de palavras do vocabulário no texto.
    Por exemplo, [1, 0, 0, 1, 1, 0].
    ```

* output_mode='count':
    ```
    Conta a frequência de cada palavra no vocabulário.
    Um texto pode ser transformado em um vetor como
    [1, 0, 0, 2, 1, 0] indicando que algumas palavras
    aparecem duas vezes.
    ```

* output_mode='tf-idf':
    ```
    Produz vetores TF-IDF, que consideram tanto a frequência
    de uma palavra em um documento quanto a frequência
    dessa palavra em todo o corpus. O output pode ser
    um vetor como [0.5, 0, 0, 0.5, 0.3, 0].
    ```

In [117]:
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int'
)

vectorizer.adapt(X_train)

In [118]:
X_train = vectorizer(X_train)
X_test = vectorizer(X_test)

In [119]:
word_index = vectorizer.get_vocabulary()

In [142]:
model = Sequential([
    Embedding(input_dim=len(word_index) + 1, output_dim=16),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [143]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=64)

In [150]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Acurácia: {accuracy * 100:.2f}%')

7/7 [==============================] - 0s 27ms/step - loss: 2.1489 - accuracy: 0.7274
Acurácia: 72.74%
